# Recursive serving

recursive_serve_flow is a util function that recursively serves all local subflows of the flow you intend to serve. This is helpful when working with deeply nested flows, as it means you don't have to manually serve individual subflows. recursive_serve_flow will fetch the default config of the flow you are trying to serve, iterate over its subflows and recursively call itself on each subflow that specifies a local `user_id` (or doesn't have user_id field). 

recursive_serve_flow expects that a subflow config will contain fields corresponding to arguments needed to call recursive_serve_flow on that subflow. This means that along with the standard `user_id` and `flow_endpoint` fields, the subflow configs should contain the `flow_class_name` field so that recursive_serve knows what to serve at that endpoint. Recursive serve will also pick up additional serve arguments from the subflow configs (`singleton`, `dispatch_point`, `parallel_dispatch`), or give them default values if missing.

In [1]:
%load_ext autoreload
%autoreload 2
import os, json
from colink import CoLink
from aiflows.utils import serving
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread

/Users/nicolasbaldwin/opt/miniconda3/envs/mockenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cl = colink_utils.start_colink_server()

### Start a few default workers

In [3]:
run_dispatch_worker_thread(cl)
run_dispatch_worker_thread(cl)

[2024-04-08 10:33:20,303][aiflows.workers.dispatch_worker:236][INFO] - Dispatch worker started in attached thread.
[2024-04-08 10:33:20,315][aiflows.workers.dispatch_worker:237][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:33:20,322][aiflows.workers.dispatch_worker:236][INFO] - Dispatch worker started in attached thread.
[2024-04-08 10:33:20,329][aiflows.workers.dispatch_worker:237][INFO] - dispatch_point: coflows_dispatch


### Observe default config of the Flow we will serve

Note the `flow_class_name` and `singleton` fields of the first_reverse_flow subflow. 

In [4]:
cfg = read_yaml_file("ReverseNumberFlowModule/ReverseNumberSequentialFlow.yaml")
print(json.dumps(cfg, indent=4))

{
    "name": "ReverseNumberTwice",
    "description": "A sequential flow that reverses a number twice.",
    "_target_": "ReverseNumberFlowModule.ReverseNumberSequentialFlow.instantiate_from_default_config",
    "input_interface": [
        "number"
    ],
    "output_interface": [
        "output_number"
    ],
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": "local",
            "flow_endpoint": "reverse_number_atomic",
            "flow_class_name": "ReverseNumberFlowModule.ReverseNumberAtomicFlow",
            "singleton": true,
            "name": "ReverseNumberFirst",
            "description": "A flow that takes in a number and reverses it."
        },
        "second_reverse_flow": {
            "user_id": "local",
            "flow_endpoint": "reverse_number_atomic",
            "flow_class_name": "ReverseNumberFlowModule.ReverseNumberAtomicFlow",
            "name": "ReverseNumberSecond",
            "description": "A flow that takes in a numbe

### Call recursive serve

In [5]:
serving.recursive_serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential"
)
# ReverseNumberAtomicFlow gets automatically served

[2024-04-08 10:33:25,610][aiflows.utils.serving:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
[2024-04-08 10:33:25,612][aiflows.utils.serving:117][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:33:25,613][aiflows.utils.serving:118][INFO] - parallel_dispatch: False
[2024-04-08 10:33:25,615][aiflows.utils.serving:119][INFO] - singleton: True

[2024-04-08 10:33:25,620][aiflows.utils.serving:716][INFO] - Subflow second_reverse_flow already served.
[2024-04-08 10:33:25,651][aiflows.utils.serving:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
[2024-04-08 10:33:25,652][aiflows.utils.serving:117][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:33:25,653][aiflows.utils.serving:118][INFO] - parallel_dispatch: False
[2024-04-08 10:33:25,655][aiflows.utils.serving:119][INFO] - singleton: False



True

### Get instance

In [6]:
flow = serving.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_sequential",
    user_id="local",
)

[2024-04-08 10:33:27,314][aiflows.utils.serving:336][INFO] - Mounted ba8eb583-ef67-4be4-943c-8e4785044127 at flows:reverse_number_atomic:mounts:local:ba8eb583-ef67-4be4-943c-8e4785044127
[2024-04-08 10:33:27,336][aiflows.utils.serving:543][INFO] - Fetched singleton ba8eb583-ef67-4be4-943c-8e4785044127
[2024-04-08 10:33:27,387][aiflows.utils.serving:336][INFO] - Mounted e13b2394-e19e-4ac4-a641-abf32335ef14 at flows:reverse_number_sequential:mounts:local:e13b2394-e19e-4ac4-a641-abf32335ef14


In [7]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
reply_data = flow.get_reply_future(input_message).get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

[2024-04-08 10:33:28,461][aiflows.workers.dispatch_worker:119][INFO] - 
~~~ Dispatch task ~~~
[2024-04-08 10:33:28,470][aiflows.workers.dispatch_worker:161][INFO] - flow_endpoint: reverse_number_sequential
[2024-04-08 10:33:28,473][aiflows.workers.dispatch_worker:162][INFO] - flow_id: e13b2394-e19e-4ac4-a641-abf32335ef14
[2024-04-08 10:33:28,475][aiflows.workers.dispatch_worker:163][INFO] - owner_id: local
[2024-04-08 10:33:28,478][aiflows.workers.dispatch_worker:164][INFO] - message_paths: ['push_tasks:a5ff94e7-9b2b-4c77-9515-960a39dd9a9e:msg']
[2024-04-08 10:33:28,481][aiflows.workers.dispatch_worker:165][INFO] - parallel_dispatch: False

[2024-04-08 10:33:28,551][aiflows.workers.dispatch_worker:188][INFO] - Input message source: Proxy_reverse_number_sequential
Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}
[2024-04-08 10:33:28,604][aiflows.workers.dispatch_worker:119][INFO] - 
~~~ Dispatch task ~~~
[2024-04-08 10:33:28,617][aiflows.workers.dispatch_wor

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 

